In [1]:
# Imports
import itertools
from itertools import permutations
import os
import platform
import random
from tkinter import Tk

from cvxopt import solvers, matrix
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import torch
import torchvision
from torchvision import transforms, models,datasets
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# OS related settings
if platform.system() == 'Windows':
    print('Windows')
#     %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    print('macOS')
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    print('Linux')
# This line of "print" must exist right after %matplotlib command, otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')

macOS
Interactive plot activated


In [2]:
# ChIQP

# Silencing solvers.qp

from contextlib import redirect_stdout
from io import StringIO
class NullIO(StringIO):
    def write(self, txt):
        pass


def silent(fn):
    """Decorator to silence functions."""
    def silent_fn(*args, **kwargs):
        with redirect_stdout(NullIO()):
            return fn(*args, **kwargs)
    return silent_fn






class ChoquetIntegral:

    def __init__(self):
        """Instantiation of a ChoquetIntegral.
           This sets up the ChI. It doesn't take any input parameters
           because you may want to use pass your own values in(as opposed
           to learning from data). To instatiate, use
           chi = ChoquetIntegral.ChoquetIntegral()
        """
        self.trainSamples, self.trainLabels = [], []
        self.testSamples, self.testLabels = [], []
        self.N, self.numberConstraints, self.M = 0, 0, 0
        self.g = 0
        self.fm = []
        self.type = []


    def train_chi(self, x1, l1):
        """
        This trains this instance of your ChoquetIntegral w.r.t x1 and l1.
        :param x1: These are the training samples of size N x M(inputs x number of samples)
        :param l1: These are the training labels of size 1 x M(label per sample)
        """
        self.type = 'quad'
        self.trainSamples = x1
        self.trainLabels = l1
        self.N = self.trainSamples.shape[0]
        self.M = self.trainSamples.shape[1]
#         print("Number Inputs : ", self.N, "; Number Samples : ", self.M)
        self.fm = self.produce_lattice()

        return self



    def chi_quad(self, x2):
        """
        This will produce an output for this instance of the ChI
        This will use the learned(or specified) Choquet integral to
        produce an output w.r.t. to the new input.
        :param x2: testing sample
        :return: output of the choquet integral.
        """
        if self.type == 'quad':
            n = len(x2)
            pi_i = np.argsort(x2)[::-1][:n] + 1
            ch = x2[pi_i[0] - 1] * (self.fm[str(pi_i[:1])])
            for i in range(1, n):
                latt_pti = np.sort(pi_i[:i + 1])
                latt_ptimin1 = np.sort(pi_i[:i])
                ch = ch + x2[pi_i[i] - 1] * (self.fm[str(latt_pti)] - self.fm[str(latt_ptimin1)])
            return ch
        else:
            print("If using sugeno measure, you need to use chi_sugeno.")


    def produce_lattice(self):
        """
            This method builds is where the lattice(or FM variables) will be learned.
            The FM values can be found via a quadratic program, which is used here
            after setting up constraint matrices. Refer to papers for complete overview.
        :return: Lattice, the learned FM variables.
        """

        fm_len = 2 ** self.N - 1  # nc
        E = np.zeros((fm_len, fm_len))  # D
        L = np.zeros(fm_len)  # f
        index_keys = self.get_keys_index()
        for i in range(0, self.M):  # it's going through one sample at a time.
            l = self.trainLabels[i]  # this is the labels
            fm_coeff = self.get_fm_class_img_coeff(index_keys, self.trainSamples[:, i], fm_len)  # this is Hdiff
            # print(fm_coeff)
            L = L + (-2) * l * fm_coeff
            E = E + np.matmul(fm_coeff.reshape((fm_len, 1)), fm_coeff.reshape((1, fm_len)))

        G, h, A, b = self.build_constraint_matrices(index_keys, fm_len)
        solvers_qp = silent(solvers.qp)
        sol = solvers_qp(matrix(2 * E, tc='d'), matrix(L.T, tc='d'), matrix(G, tc='d'), matrix(h, tc='d'),
                         matrix(A, tc='d'), matrix(b, tc='d'))
        g = sol['x']
        Lattice = {}
        for key in index_keys.keys():
            Lattice[key] = g[index_keys[key]]
        return Lattice


    def build_constraint_matrices(self, index_keys, fm_len):
        """
        This method builds the necessary constraint matrices.
        :param index_keys: map to reference lattice components
        :param fm_len: length of the fuzzy measure
        :return: the constraint matrices
        """

        vls = np.arange(1, self.N + 1)
        line = np.zeros(fm_len)
        G = line
        line[index_keys[str(np.array([1]))]] = -1.
        h = np.array([0])
        for i in range(2, self.N + 1):
            line = np.zeros(fm_len)
            line[index_keys[str(np.array([i]))]] = -1.
            G = np.vstack((G, line))
            h = np.vstack((h, np.array([0])))
        for i in range(2, self.N + 1):
            parent = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in parent:
                for j in range(len(latt_pt) - 1, len(latt_pt)):
                    children = np.array(list(itertools.combinations(latt_pt, j)))
                    for latt_ch in children:
                        line = np.zeros(fm_len)
                        line[index_keys[str(latt_ch)]] = 1.
                        line[index_keys[str(latt_pt)]] = -1.
                        G = np.vstack((G, line))
                        h = np.vstack((h, np.array([0])))

        line = np.zeros(fm_len)
        line[index_keys[str(vls)]] = 1.
        G = np.vstack((G, line))
        h = np.vstack((h, np.array([1])))

        # equality constraints
        A = np.zeros((1, fm_len))
        A[0, -1] = 1
        b = np.array([1]);

        return G, h, A, b


    def get_fm_class_img_coeff(self, Lattice, h, fm_len):  # Lattice is FM_name_and_index, h is the samples, fm_len
        """
        This creates a FM map with the name as the key and the index as the value
        :param Lattice: dictionary with FM
        :param h: sample
        :param fm_len: fm length
        :return: the fm_coeff
        """

        n = len(h)  # len(h) is the number of the samples
        fm_coeff = np.zeros(fm_len)
        pi_i = np.argsort(h)[::-1][:n] + 1
        for i in range(1, n):
            fm_coeff[Lattice[str(np.sort(pi_i[:i]))]] = h[pi_i[i - 1] - 1] - h[pi_i[i] - 1]
        fm_coeff[Lattice[str(np.sort(pi_i[:n]))]] = h[pi_i[n - 1] - 1]
        np.matmul(fm_coeff, np.transpose(fm_coeff))
        return fm_coeff


    def get_keys_index(self):
        """
        Sets up a dictionary for referencing FM.
        :return: The keys to the dictionary
        """

        vls = np.arange(1, self.N + 1)
        count = 0
        Lattice = {}
        for i in range(0, self.N):
            Lattice[str(np.array([vls[i]]))] = count
            count = count + 1
        for i in range(2, self.N + 1):
            A = np.array(list(itertools.combinations(vls, i)))
            for latt_pt in A:
                Lattice[str(latt_pt)] = count
                count = count + 1
        return Lattice



In [ ]:
# All in one


# Parameters
num_epoch = 100
dim_list = list(range(5, 6))
num_per_perm_train = 10 # Every permutation gets the same number of train/test data samples
num_per_perm_test = 100 # Every permutation gets the same number of train/test data samples
eva_func_num = 4
superset_factor = 5

SSEs_avg_list = [] # Avg SSE with seen data of every dim
SSEs_c_avg_list = [] # Avg SSE with unseen data of every dim

# Experiment for dim = 3 to 8
for dim in dim_list:
    
    # Every permutation gets the same number of train/test data samples,
    # To ensure that, calculate the number of data needed in total, 
    # and generate a dataset that is multiple times bigger.
    num_train = math.factorial(dim) * num_per_perm_train
    num_test = math.factorial(dim) * num_per_perm_test
    # Create superset
    train_data_superset = np.random.rand(dim, num_train*superset_factor)
    test_data_superset = np.random.rand(dim, num_test*superset_factor)
    # Get permutation of each data sample
    train_data_perms = np.argsort(train_data_superset, 0)
    test_data_perms = np.argsort(test_data_superset, 0)
    # N! possible permutations
    all_perms = list(permutations(list(range(dim))))
    # Group data sample according to its permutation
    train_data_superset_by_perm = []
    test_data_superset_by_perm = []
    for i, current_perm in enumerate(all_perms):
        # Get index of data sample of certain permutation and save to list
        temp = np.where(train_data_perms[0, :]==current_perm[0])
        for j, idx in enumerate(current_perm):
            temp = np.intersect1d(temp, np.where(train_data_perms[j, :]==idx))
        if temp.size < num_per_perm_train:
            print('Current permutation doesn\'t have sufficient number of samples. Please regenerate!')
            exit()
        train_data_superset_by_perm.append(temp)
        
        # Get index of data sample of certain permutation and save to list
        temp = np.where(test_data_perms[0, :]==current_perm[0])
        for j, idx in enumerate(current_perm):
            temp = np.intersect1d(temp, np.where(test_data_perms[j, :]==idx))
        if temp.size < num_per_perm_test:
            print('Current permutation doesn\'t have sufficient number of samples. Please regenerate!')
            exit()
        test_data_superset_by_perm.append(temp)
    print('Data ready')
    print('Train data superset size:', train_data_superset.shape)
    print('Test data superset size:', test_data_superset.shape)



    SSEs_sum = np.zeros((len(all_perms), eva_func_num)) # Sum of SSE with seen data of certain dim
    SSEs_c_sum = np.zeros((len(all_perms)-1, eva_func_num)) # Sum of SSE with unseen data of certain dim

    for epoch in range(num_epoch):
        print('Epoch', epoch)
        
        # Every permutation gets the same number of train/test data samples,
        # Data is randomly pull from superset each epoch
        train_data_by_perm = []
        test_data_by_perm = []
        for i in range(len(all_perms)):
            temp = train_data_superset_by_perm[i]
            random.shuffle(temp)
            train_data_by_perm.append(temp[0:num_per_perm_train])

            temp = test_data_superset_by_perm[i]
            random.shuffle(temp)
            test_data_by_perm.append(temp[0:num_per_perm_test])

        SSEs = np.zeros((len(all_perms), eva_func_num))
        SSEs_c = np.zeros((len(all_perms)-1, eva_func_num))
        
        # Train group limit
        train_group_num_limit = math.factorial(5)
        step = 1
        if len(all_perms) > train_group_num_limit:
            step = int(len(all_perms) / train_group_num_limit)
        for i in tqdm(range(step-1, len(all_perms), step)):

            train_idx = np.concatenate(train_data_by_perm[0:i+1])
            test_idx = np.concatenate(test_data_by_perm[0:i+1])
            
            train_d = train_data_superset[:, train_idx]
            test_d = test_data_superset[:, test_idx]

            train_label_min = np.amin(train_d, 0)
            train_label_max = np.amax(train_d, 0)
            train_label_mean = np.mean(train_d, 0)
            train_label_gmean = np.cbrt(np.prod(train_d, 0))

            test_label_min = np.amin(test_d, 0)
            test_label_max = np.amax(test_d, 0)
            test_label_mean = np.mean(test_d, 0)
            test_label_gmean = np.cbrt(np.prod(test_d, 0))

            if i < len(all_perms)-1:
                test_idx_c = np.concatenate(test_data_by_perm[i:-1])
                test_d_c = test_data_superset[:, test_idx_c]

                test_label_min_c = np.amin(test_d_c, 0)
                test_label_max_c = np.amax(test_d_c, 0)
                test_label_mean_c = np.mean(test_d_c, 0)
                test_label_gmean_c = np.cbrt(np.prod(test_d_c, 0))


            chi = ChoquetIntegral()
            chi.train_chi(train_d, train_label_min)
            SSE = 0
            for j in range(np.size(test_d, 1)):
                chi_min_test = chi.chi_quad(test_d[:, j])
                SSE += (chi_min_test - test_label_min[j]) ** 2
            SSEs[i, 0] = SSE / np.size(test_d)
    #         print('\nChi min', chi.fm)
    #         print('\nSSE', SSE)
            if i < len(all_perms)-1:
                SSE_c = 0
                for j in range(np.size(test_d_c, 1)):
                    chi_min_test_c = chi.chi_quad(test_d_c[:, j])
                    SSE_c += (chi_min_test_c - test_label_min_c[j]) ** 2
                SSEs_c[i, 0] = SSE_c / np.size(test_d_c)
    #             print('\nUnseen data SSE', SSE_c)


            chi = ChoquetIntegral()
            chi.train_chi(train_d, train_label_max)
            SSE = 0
            for j in range(np.size(test_d, 1)):
                chi_max_test = chi.chi_quad(test_d[:, j])
                SSE += (chi_max_test - test_label_max[j]) ** 2
            SSEs[i, 1] = SSE / np.size(test_d)
    #         print('\nChi max', chi.fm)
    #         print('\nSSE', SSE)
            if i < len(all_perms)-1:
                SSE_c = 0
                for j in range(np.size(test_d_c, 1)):
                    chi_max_test_c = chi.chi_quad(test_d_c[:, j])
                    SSE_c += (chi_max_test_c - test_label_max_c[j]) ** 2
                SSEs_c[i, 1] = SSE_c / np.size(test_d_c)
    #             print('\nUnseen data SSE', SSE_c)


            chi = ChoquetIntegral()
            chi.train_chi(train_d, train_label_mean)
            SSE = 0
            for j in range(np.size(test_d, 1)):
                chi_mean_test = chi.chi_quad(test_d[:, j])
                SSE += (chi_mean_test - test_label_mean[j]) ** 2
            SSEs[i, 2] = SSE / np.size(test_d)
    #         print('\nChi mean', chi.fm)
    #         print('\nSSE', SSE)
            if i < len(all_perms)-1:
                SSE_c = 0
                for j in range(np.size(test_d_c, 1)):
                    chi_mean_test_c = chi.chi_quad(test_d_c[:, j])
                    SSE_c += (chi_mean_test_c - test_label_mean_c[j]) ** 2
                SSEs_c[i, 2] = SSE_c / np.size(test_d_c)
    #             print('\nUnseen data SSE', SSE_c)


            chi = ChoquetIntegral()
            chi.train_chi(train_d, train_label_gmean)
            SSE = 0
            for j in range(np.size(test_d, 1)):
                chi_gmean_test = chi.chi_quad(test_d[:, j])
                SSE += (chi_gmean_test - test_label_gmean[j]) ** 2
            SSEs[i, 3] = SSE / np.size(test_d)
    #         print('\nChi geometric mean', chi.fm)
    #         print('\nSSE', SSE)
            if i < len(all_perms)-1:
                SSE_c = 0
                for j in range(np.size(test_d_c, 1)):
                    chi_gmean_test_c = chi.chi_quad(test_d_c[:, j])
                    SSE_c += (chi_gmean_test_c - test_label_gmean_c[j]) ** 2
                SSEs_c[i, 3] = SSE_c / np.size(test_d_c)
    #             print('\nUnseen data SSE', SSE_c)


    #         print('\n\n\n')

            SSEs_sum += SSEs
            SSEs_c_sum += SSEs_c


    SSEs_avg = SSEs_sum / num_epoch
    SSEs_c_avg = SSEs_c_sum / num_epoch
    
    SSEs_avg_list.append(SSEs_avg)
    SSEs_c_avg_list.append(SSEs_c_avg)

120


  0%|          | 0/120 [00:00<?, ?it/s]

Data ready
Train data superset size: (5, 6000)
Test data superset size: (5, 60000)
Epoch 0


  8%|▊         | 10/120 [02:42<29:36, 16.15s/it]

In [20]:
# 2, 6, 24, 120, 720
num = 24
step = 1
if num > 6:
    step = int(num / 6)
for i in range(step-1, num, step):
    print(i)
for i in range(0, num):
    print(i)

3
7
11
15
19
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [ ]:
# Plot error

for i, (dim, SSEs_avg, SSEs_c_avg) in enumerate(zip(dim_list, SSEs_avg_list, SSEs_c_avg_list)):
    num_perm = math.factorial(dim)
    fig, ax = plt.subplots()
    x = (np.asarray(list(range(1, num_perm+1)))) / num_perm
    print(x.shape, SSEs_avg.shape)
    print(x)
#     print(SSEs_avg)
    plt.plot(x, SSEs_avg)
    ax.set_title('SSE (Data with same pattern)')
    ax.legend(['Min', 'Max', 'Mean', 'Geometric Mean'])
    ax.set_xlabel('Percentage of Seen Data')
    ax.set_ylabel('SSEs avg')
    ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

    plt.show()



    fig, ax = plt.subplots()
    x = (np.asarray(list(range(1, num_perm)))) / num_perm
    print(x.shape, SSEs_c_avg.shape)
    print(x)
#     print(SSEs_c_avg)
    plt.plot(x, SSEs_c_avg)
    ax.set_title('SSE (Data with unseen pattern)')
    ax.legend(['Min', 'Max', 'Mean', 'Geometric Mean'])
    ax.set_xlabel('Percentage of Seen Data')
    ax.set_ylabel('SSEs avg')
    ax.xaxis.set_major_formatter(FuncFormatter('{0:.0%}'.format))

    plt.show()